# Урок 3. Логистическая регрессия. Log Loss.

### Задание-1:
Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.  

In [19]:
import numpy as np
import matplotlib.pyplot as plt

In [20]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [21]:
def calc_logloss(y, y_pred):
    y_pred_ = np.clip(y_pred, 1e-16, 1)
    err = - np.mean(y * np.log(y_pred_) + (1.0 - y) * np.log(1.0 - y_pred_))
    err = np.sum(err)
    return err

### Задание-2:
Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [22]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [23]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

X_st

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  3.        ,
         0.        ,  5.        , 10.        ,  1.        ,  2.        ],
       [-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
        -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  2.        ,
         1.        ,  3.        ,  3.        ,  1.        ,  2.        ]])

In [24]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [28]:
W = eval_model(X_st, y, iterations=10**3, alpha=3)

100 [-8.51673764 -1.71111938 -1.38606983  7.28158726] 0.29317933648038047
200 [-13.11264893  -2.91758836  -1.32887393  15.25013166] 1.779334266776606
300 [-16.34784194  -2.52358491  -3.58710214  14.0649044 ] 0.23209165779711988
400 [-17.9180478   -2.48236136  -4.215914    15.04778717] 0.20804906807774284
500 [-19.22367205  -2.56735548  -4.65327841  16.09142534] 0.19997872978473893
600 [-20.41984797  -2.5623619   -5.04609253  17.01262184] 0.1879798019849792
700 [-21.51452025  -2.58394493  -5.39959361  17.86189317] 0.17921032739665127
800 [-22.52683447  -2.60985149  -5.72083214  18.65211697] 0.17189744297211426
900 [-23.47006092  -2.63737709  -6.01607681  19.39263488] 0.16564286047389815
1000 [-24.35402267  -2.66399475  -6.28979022  20.09126333] 0.16018416765029295


### Задание-3:
Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [29]:
def calc_pred_proba(W, X):
    return sigmoid(np.dot(X.T, W))

In [30]:
calc_pred_proba(W, X_st)

array([0.31748792, 0.03358156, 0.99999851, 0.11279739, 0.84225541,
       0.11996466, 0.99999986, 0.01770817, 0.47084048, 0.99996194])